# 라이브러리 import

In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import json, re
import plotly.express as px

# 1.ev_app_resident.csv

In [ ]:
resident = pd.read_csv('/content/drive/MyDrive/딥러닝/프로젝트/파일/1.ev_app_resident.csv', encoding='utf-8')
resident.head()

,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
0,20220605,7,4146,용인시,41463572,동백1동,86135034,127.15225,37.286300,MALE,6,EV라운지,5,3.522
1,20220625,6,4146,용인시,41465580,상현2동,84375196,127.09008,37.309536,MALE,8,파워큐브차저,2,4.231
2,20220614,2,4146,용인시,41461250,포곡읍,87958893,127.21660,37.274303,FEMALE,14,EV라운지,1,18.440
3,20220619,7,4146,용인시,41465530,신봉동,84119243,127.08097,37.323480,MALE,13,EV라운지,2,4.335
4,20220624,5,4146,용인시,41465580,상현2동,84423187,127.09178,37.306840,MALE,8,EV라운지,4,4.231


## 거주자 시간 (19시 ~ 익일 9시)

- 데이터 time_zone = 1, 2, 5, 6
- 평일 / 주말 두가지의 지도로 나타냄

### 평일

- DOW(요일) : 1, 2, 3, 4, 5

In [ ]:
resident_wd = resident[(resident['time_zone'] != 3) & (resident['time_zone'] != 4) & (resident['dow'] < 6)]
resident_wd.head()

,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
2,20220614,2,4146,용인시,41461250,포곡읍,87958893,127.21660,37.274303,FEMALE,14,EV라운지,1,18.440
8,20220620,1,4146,용인시,41461520,역삼동,86678895,127.17163,37.239033,FEMALE,11,EV라운지,5,7.258
13,20220617,5,4146,용인시,41465560,동천동,84583242,127.09727,37.336150,MALE,9,EV라운지,2,4.374
17,20220615,3,4146,용인시,41461256,이동읍,87734718,127.20903,37.189102,MALE,5,하이차저,1,3.229
18,20220621,2,4146,용인시,41461520,역삼동,87094856,127.18632,37.233210,MALE,12,EV라운지,2,4.256


#### adng_nm 기준으로 그룹화

- geojson 동기준으로 나누어져있음 -> 이 파일도 동기준으로 그룹화하여 카운트
- 어느동에서 사용자가 앱을 많이 실행했는지 알 수 있는 지표가 됨 

In [ ]:
resident_wd_group = resident_wd.groupby('adng_nm')['count_cust'].sum()
resident_wd_group = resident_wd_group.reset_index()
resident_wd_group = resident_wd_group.rename(columns={'count_cust' : 'count_cust_sum'})
resident_wd_group = resident_wd_group[['count_cust_sum']]
resident_wd_group.head()

,count_cust_sum
0,27074.658
1,24240.361
2,9175.687
3,11292.471
4,18703.483


#### resident_1 데이터 중복 제거

- adng_nm 기준으로 데이터 중복 제거

In [ ]:
resident_wd_col = resident_wd.drop_duplicates(['adng_nm'])
resident_wd_col = resident_wd_col.sort_values('adng_nm')
resident_wd_col = resident_wd_col.reset_index()
resident_wd_col.head()

,index,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
0,53,20220623,4,4146,용인시,41463520,구갈동,85063092,127.114480,37.282127,MALE,6,EV라운지,5,3.411
1,68,20220603,5,4146,용인시,41463560,구성동,85351103,127.124565,37.295230,MALE,9,EV라운지,2,3.996
2,40,20220627,1,4146,용인시,41463540,기흥동,84870993,127.107970,37.232080,MALE,12,EV라운지,5,7.824
3,55,20220624,5,4146,용인시,41461259,남사읍,86710708,127.173120,37.155655,MALE,7,EV라운지,6,4.610
4,26,20220616,4,4146,용인시,41463572,동백1동,86183017,127.153970,37.279995,FEMALE,10,EV라운지,5,3.132


#### 병합

- 그룹화한 데이터프레임과 중복제거한 데이터프레임 붙이기

In [ ]:
resident_wd_col['count_cust_sum'] = resident_wd_group
resident_wd_col.head()

,index,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust,count_cust_sum
0,53,20220623,4,4146,용인시,41463520,구갈동,85063092,127.114480,37.282127,MALE,6,EV라운지,5,3.411,27074.658
1,68,20220603,5,4146,용인시,41463560,구성동,85351103,127.124565,37.295230,MALE,9,EV라운지,2,3.996,24240.361
2,40,20220627,1,4146,용인시,41463540,기흥동,84870993,127.107970,37.232080,MALE,12,EV라운지,5,7.824,9175.687
3,55,20220624,5,4146,용인시,41461259,남사읍,86710708,127.173120,37.155655,MALE,7,EV라운지,6,4.610,11292.471
4,26,20220616,4,4146,용인시,41463572,동백1동,86183017,127.153970,37.279995,FEMALE,10,EV라운지,5,3.132,18703.483


#### cell_id, time_zone기준으로 중복 제거

- 한 곳에 여러개가 찍혀도 확인을 할 수 없기 때문에 time_zone 별 한 곳에 하나만 찍히게 하기 위해서 중복 제거

In [ ]:
resident_wd_ping = resident_wd.drop_duplicates(['cell_id', 'time_zone'])
resident_wd_ping = resident_wd_ping.reset_index()
resident_wd_ping.head()

,index,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
0,2,20220614,2,4146,용인시,41461250,포곡읍,87958893,127.21660,37.274303,FEMALE,14,EV라운지,1,18.440
1,8,20220620,1,4146,용인시,41461520,역삼동,86678895,127.17163,37.239033,FEMALE,11,EV라운지,5,7.258
2,13,20220617,5,4146,용인시,41465560,동천동,84583242,127.09727,37.336150,MALE,9,EV라운지,2,4.374
3,17,20220615,3,4146,용인시,41461256,이동읍,87734718,127.20903,37.189102,MALE,5,하이차저,1,3.229
4,18,20220621,2,4146,용인시,41461520,역삼동,87094856,127.18632,37.233210,MALE,12,EV라운지,2,4.256


#### geojson 로드

In [ ]:
path = '/content/drive/MyDrive/딥러닝/프로젝트/파일/HangJeongDong_ver20220401.geojson'

In [ ]:
rfile = open(path, 'r', encoding='utf8').read()
jsonfile =json.loads(rfile)

json_yongin = {'type': 'FeatureCollection'}
json_pick = []
json_dong = []
for item in jsonfile['features']:
    # print(item)
    if item['properties']['sggnm']=='용인시처인구' or item['properties']['sggnm']=='용인시수지구' or item['properties']['sggnm']=='용인시기흥구' :
        # print(item)
        dong = item['properties']['adm_nm'].split()[-1]

        # print(dong)
        item['id'] = dong
        # print(item)
        json_dong.append(dong)
        json_pick.append(item)
json_yongin['features']=json_pick

#### 시각화

In [ ]:
map = folium.Map((37.240608, 127.1772935), zoom_start=11)
choropleth = folium.Choropleth(json_yongin,
                  data=resident_wd_col,
                  columns=['adng_nm', 'count_cust_sum'],
                  key_on='feature.id').add_to(map)


choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['adm_nm'], lables=False)
)

for i in resident_wd_ping.index:
    sub_lat = resident_wd_ping.loc[i, 'cell_ycrd']
    sub_long = resident_wd_ping.loc[i, 'cell_xcrd']
    
    title = f"{resident_wd_ping.loc[i, 'cell_ycrd']} - {resident_wd_ping.loc[i, 'cell_xcrd']} - {resident_wd_ping.loc[i, 'adng_nm']}"
    
    color='lime'
    size=2
    if resident_wd_ping.loc[i, 'time_zone'] == 2:
        color='seagreen'
        size=4

    elif resident_wd_ping.loc[i, 'time_zone'] == 5:
      color='darkorange'
      size=6

    elif resident_wd_ping.loc[i, 'time_zone'] == 6:
      color='mediumpurple'
      size=8
    
    folium.CircleMarker(location=[sub_lat, sub_long],
                        radius=size,
                        color=color,
                        tooltip=title).add_to(map)

map

Output hidden; open in https://colab.research.google.com to view.

### 주말

- DOW(요일) : 6, 7

In [ ]:
resident_wk = resident[(resident['time_zone'] != 3) & (resident['time_zone'] != 4) & (resident['dow'] > 5)]
resident_wk.head()

,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
0,20220605,7,4146,용인시,41463572,동백1동,86135034,127.152250,37.286300,MALE,6,EV라운지,5,3.522
1,20220625,6,4146,용인시,41465580,상현2동,84375196,127.090080,37.309536,MALE,8,파워큐브차저,2,4.231
3,20220619,7,4146,용인시,41465530,신봉동,84119243,127.080970,37.323480,MALE,13,EV라운지,2,4.335
6,20220612,7,4146,용인시,41463530,상갈동,84855076,127.107216,37.269030,MALE,12,EV라운지,5,7.448
15,20220625,6,4146,용인시,41463530,상갈동,84839073,127.106660,37.267227,FEMALE,10,EV라운지,2,6.246


#### adng_nm 기준으로 그룹화

- geojson 동기준으로 나누어져있음 -> 이 파일도 동기준으로 그룹화하여 카운트
- 어느동에서 사용자가 앱을 많이 실행했는지 알 수 있는 지표가 됨 

In [ ]:
resident_wk_group = resident_wk.groupby('adng_nm')['count_cust'].sum()
resident_wk_group = resident_wk_group.reset_index()
resident_wk_group = resident_wk_group.rename(columns={'count_cust' : 'count_cust_sum'})
resident_wk_group = resident_wk_group[['count_cust_sum']]
resident_wk_group.head()

,count_cust_sum
0,8805.607
1,8251.461
2,3331.441
3,3976.654
4,6782.071


#### resident_wk 데이터 중복 제거

- adng_nm 기준으로 데이터 중복 제거


In [ ]:
resident_wk_col = resident_wk.drop_duplicates(['adng_nm'])
resident_wk_col = resident_wk_col.sort_values('adng_nm')
resident_wk_col = resident_wk_col.reset_index()
resident_wk_col.head()

,index,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
0,99,20220619,7,4146,용인시,41463520,구갈동,85047089,127.113920,37.280323,MALE,9,EV라운지,6,3.048
1,35,20220605,7,4146,용인시,41463560,구성동,85703079,127.136986,37.294370,FEMALE,9,EV라운지,5,8.098
2,1009,20220611,6,4146,용인시,41463540,기흥동,85270989,127.122020,37.241590,MALE,12,하이차저,5,3.912
3,2111,20220618,6,4146,용인시,41461259,남사읍,86790709,127.175930,37.158367,MALE,10,EV라운지,6,4.760
4,0,20220605,7,4146,용인시,41463572,동백1동,86135034,127.152250,37.286300,MALE,6,EV라운지,5,3.522


#### 병합

- 그룹화한 데이터프레임과 중복제거한 데이터프레임 붙이기

In [ ]:
resident_wk_col['count_cust_sum'] = resident_wk_group
resident_wk_col.head()

,index,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust,count_cust_sum
0,99,20220619,7,4146,용인시,41463520,구갈동,85047089,127.113920,37.280323,MALE,9,EV라운지,6,3.048,8805.607
1,35,20220605,7,4146,용인시,41463560,구성동,85703079,127.136986,37.294370,FEMALE,9,EV라운지,5,8.098,8251.461
2,1009,20220611,6,4146,용인시,41463540,기흥동,85270989,127.122020,37.241590,MALE,12,하이차저,5,3.912,3331.441
3,2111,20220618,6,4146,용인시,41461259,남사읍,86790709,127.175930,37.158367,MALE,10,EV라운지,6,4.760,3976.654
4,0,20220605,7,4146,용인시,41463572,동백1동,86135034,127.152250,37.286300,MALE,6,EV라운지,5,3.522,6782.071


#### cell_id, time_zone기준으로 중복 제거

- 한 곳에 여러개가 찍혀도 확인을 할 수 없기 때문에 time_zone 별 한 곳에 하나만 찍히게 하기 위해서 중복 제거

In [ ]:
resident_wk_ping = resident_wk.drop_duplicates(['cell_id', 'time_zone'])
resident_wk_ping = resident_wk_ping.reset_index()
resident_wk_ping.head()

,index,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
0,0,20220605,7,4146,용인시,41463572,동백1동,86135034,127.152250,37.286300,MALE,6,EV라운지,5,3.522
1,1,20220625,6,4146,용인시,41465580,상현2동,84375196,127.090080,37.309536,MALE,8,파워큐브차저,2,4.231
2,3,20220619,7,4146,용인시,41465530,신봉동,84119243,127.080970,37.323480,MALE,13,EV라운지,2,4.335
3,6,20220612,7,4146,용인시,41463530,상갈동,84855076,127.107216,37.269030,MALE,12,EV라운지,5,7.448
4,15,20220625,6,4146,용인시,41463530,상갈동,84839073,127.106660,37.267227,FEMALE,10,EV라운지,2,6.246


#### 시각화

In [ ]:
map = folium.Map((37.240608, 127.1772935), zoom_start=11)
choropleth = folium.Choropleth(json_yongin,
                  data=resident_wk_col,
                  columns=['adng_nm', 'count_cust_sum'],
                  key_on='feature.id').add_to(map)


choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['adm_nm'], lables=False)
)

for i in resident_wk_ping.index:
    sub_lat = resident_wk_ping.loc[i, 'cell_ycrd']
    sub_long = resident_wk_ping.loc[i, 'cell_xcrd']
    
    title = f"{resident_wk_ping.loc[i, 'cell_ycrd']} - {resident_wk_ping.loc[i, 'cell_xcrd']} - {resident_wk_ping.loc[i, 'adng_nm']}"
    
    color='lime'
    size=2
    if resident_wk_ping.loc[i, 'time_zone'] == 2:
        color='seagreen'
        size=4

    elif resident_wk_ping.loc[i, 'time_zone'] == 5:
      color='darkorange'
      size=6

    elif resident_wk_ping.loc[i, 'time_zone'] == 6:
      color='mediumpurple'
      size=8
    
    folium.CircleMarker(location=[sub_lat, sub_long],
                        radius=size,
                        color=color,
                        tooltip=title).add_to(map)

map

Output hidden; open in https://colab.research.google.com to view.

# 용인시 아파트 현황

In [ ]:
apt = pd.read_csv('/content/drive/MyDrive/딥러닝/프로젝트/파일/용인시아파트정보.csv',encoding='utf-8')
apt.head()

,구,동,구분,단지명,세대수,사용검사일,도로명주소,Latitude,Longitude
0,수지구,죽전동,임대,용인 죽전 경기행복주택,85,20220826,수지구 대지로 37,37.329063,127.124858
1,수지구,신봉동,분양,수지스카이뷰푸르지오,363,20220628,수지구 신봉3로7번길 31,37.326492,127.065772
2,수지구,신봉동,분양,힐스테이트 광교산,789,20220429,수지구 신봉2로 154,37.328865,127.072204
3,수지구,동천동,분양,한화 포레나 수지동천,293,20220221,수지구 고기로45번길 40,37.341428,127.090150
4,기흥구,언남동,분양,힐스테이트 구성,699,(20210731~20230730),기흥구 용인향교로 29,37.296737,127.120276


In [ ]:
map = folium.Map(location=[37.240608,127.1772935],zoom_start=11)

folium.GeoJson(json_yongin).add_to(map)

for i in apt.index:
    apt1_lat = apt.loc[i,'Latitude']
    apt1_long = apt.loc[i,'Longitude']

    apt_name = f"{apt.loc[i, '단지명']}"
    apt_address = f"{apt.loc[i, '도로명주소']}"
    apt_count = f"{apt.loc[i, '세대수']}"

    if apt.loc[i,'구']=='수지구':
      color ='red'
    elif apt.loc[i,'구']=='기흥구':
      color = 'blue'
    elif apt.loc[i,'구']=='처인구':
      color = 'green'
    else:
      color = 'black'

    folium.Marker(location=[apt1_lat,apt1_long],radius=5,icon=folium.Icon(icon='home',color=color),popup=f'<pre>{apt_name}\\n 주소: {apt_address}\\n 세대수: {apt_count}</pre>').add_to(map)
map

# 2.ev_app_activity.csv

In [ ]:
activity = pd.read_csv('/content/drive/MyDrive/딥러닝/프로젝트/파일/2.ev_app_activity.csv', encoding='utf-8')
activity.head()

,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
0,20220614,2,4146,용인시,41461256,이동읍,87958702,127.216920,37.188220,MALE,12,EV라운지,4,3.829
1,20220613,1,4146,용인시,41463570,마북동,85431116,127.127350,37.303352,MALE,14,EV라운지,1,9.994
2,20220602,4,4146,용인시,41463575,동백2동,86198990,127.154590,37.268280,MALE,14,EV라운지,4,7.502
3,20220606,1,4146,용인시,41461259,남사읍,86854741,127.178110,37.174600,MALE,9,EV라운지,4,3.234
4,20220608,3,4146,용인시,41465540,죽전1동,85159183,127.117645,37.325855,FEMALE,10,EV라운지,5,3.315


## 활동지 시간 (9시 ~ 19시)

- 데이터 time_zone = 3, 4
- 평일 / 주말 두가지의 지도로 나타냄

### 평일

- DOW(요일) : 1, 2, 3, 4, 5

In [ ]:
activity_wd = activity[((activity['time_zone'] == 3) | (activity['time_zone'] == 4)) & (activity['dow'] < 6)]
activity_wd

,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
0,20220614,2,4146,용인시,41461256,이동읍,87958702,127.216920,37.188220,MALE,12,EV라운지,4,3.829
2,20220602,4,4146,용인시,41463575,동백2동,86198990,127.154590,37.268280,MALE,14,EV라운지,4,7.502
3,20220606,1,4146,용인시,41461259,남사읍,86854741,127.178110,37.174600,MALE,9,EV라운지,4,3.234
6,20220606,1,4146,용인시,41461250,포곡읍,88086871,127.221140,37.268005,FEMALE,14,EV라운지,4,23.050
12,20220621,2,4146,용인시,41463520,구갈동,85271071,127.121830,37.278545,FEMALE,8,EV라운지,4,3.596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264703,20220623,4,4146,용인시,41463550,서농동,83847056,127.071900,37.231503,MALE,7,EV라운지,4,20.073
264704,20220608,3,4146,용인시,41461259,남사읍,85878686,127.144000,37.122220,MALE,9,EV라운지,3,4.140
264707,20220624,5,4146,용인시,41463540,기흥동,84502989,127.095070,37.219864,MALE,7,EV라운지,4,3.632
264711,20220620,1,4146,용인시,41465590,성복동,83895245,127.073105,37.318040,MALE,14,EV라운지,3,11.406


#### adng_nm 기준으로 그룹화

- geojson 동기준으로 나누어져있음 -> 이 파일도 동기준으로 그룹화하여 카운트
- 어느동에서 사용자가 앱을 많이 실행했는지 알 수 있는 지표가 됨

In [ ]:
activity_wd_group = activity_wd.groupby('adng_nm')['count_cust'].sum()
activity_wd_group = activity_wd_group.reset_index()
activity_wd_group = activity_wd_group.rename(columns={'count_cust' : 'count_cust_sum'})
activity_wd_group = activity_wd_group[['count_cust_sum']]
activity_wd_group.head()

,count_cust_sum
0,21208.208
1,19561.578
2,24773.175
3,22931.620
4,13305.131


#### activity_wd 데이터 중복 제거

- adng_nm 기준으로 데이터 중복 제거


In [ ]:
activity_wd_col = activity_wd.drop_duplicates(['adng_nm'])
activity_wd_col = activity_wd_col.sort_values('adng_nm')
activity_wd_col = activity_wd_col.reset_index()
activity_wd_col.head()

,index,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
0,12,20220621,2,4146,용인시,41463520,구갈동,85271071,127.121830,37.278545,FEMALE,8,EV라운지,4,3.596
1,58,20220623,4,4146,용인시,41463560,구성동,85479099,127.129074,37.297047,FEMALE,12,EV라운지,4,8.346
2,51,20220623,4,4146,용인시,41463540,기흥동,84919021,127.109590,37.246056,MALE,7,EV라운지,4,3.594
3,3,20220606,1,4146,용인시,41461259,남사읍,86854741,127.178110,37.174600,MALE,9,EV라운지,4,3.234
4,425,20220620,1,4146,용인시,41463572,동백1동,86375016,127.160720,37.284973,FEMALE,9,EV라운지,4,8.114


#### 병합

- 그룹화한 데이터프레임과 중복제거한 데이터프레임 붙이기

In [ ]:
activity_wd_col['count_cust_sum'] = activity_wd_group
activity_wd_col.head()

,index,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust,count_cust_sum
0,12,20220621,2,4146,용인시,41463520,구갈동,85271071,127.121830,37.278545,FEMALE,8,EV라운지,4,3.596,21208.208
1,58,20220623,4,4146,용인시,41463560,구성동,85479099,127.129074,37.297047,FEMALE,12,EV라운지,4,8.346,19561.578
2,51,20220623,4,4146,용인시,41463540,기흥동,84919021,127.109590,37.246056,MALE,7,EV라운지,4,3.594,24773.175
3,3,20220606,1,4146,용인시,41461259,남사읍,86854741,127.178110,37.174600,MALE,9,EV라운지,4,3.234,22931.620
4,425,20220620,1,4146,용인시,41463572,동백1동,86375016,127.160720,37.284973,FEMALE,9,EV라운지,4,8.114,13305.131


#### cell_id, time_zone기준으로 중복 제거

- 한 곳에 여러개가 찍혀도 확인을 할 수 없기 때문에 time_zone 별 한 곳에 하나만 찍히게 하기 위해서 중복 제거

In [ ]:
activity_wd_ping = activity_wd.drop_duplicates(['cell_id', 'time_zone'])
activity_wd_ping = activity_wd_ping.reset_index()
activity_wd_ping.head()

,index,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
0,0,20220614,2,4146,용인시,41461256,이동읍,87958702,127.21692,37.188220,MALE,12,EV라운지,4,3.829
1,2,20220602,4,4146,용인시,41463575,동백2동,86198990,127.15459,37.268280,MALE,14,EV라운지,4,7.502
2,3,20220606,1,4146,용인시,41461259,남사읍,86854741,127.17811,37.174600,MALE,9,EV라운지,4,3.234
3,6,20220606,1,4146,용인시,41461250,포곡읍,88086871,127.22114,37.268005,FEMALE,14,EV라운지,4,23.050
4,12,20220621,2,4146,용인시,41463520,구갈동,85271071,127.12183,37.278545,FEMALE,8,EV라운지,4,3.596


#### 시각화

In [ ]:
map = folium.Map((37.240608, 127.1772935), zoom_start=11)
choropleth = folium.Choropleth(json_yongin,
                  data=activity_wd_col,
                  columns=['adng_nm', 'count_cust_sum'],
                  key_on='feature.id').add_to(map)


choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['adm_nm'], lables=False)
)

for i in activity_wd_ping.index:
    sub_lat = activity_wd_ping.loc[i, 'cell_ycrd']
    sub_long = activity_wd_ping.loc[i, 'cell_xcrd']
    
    title = f"{activity_wd_ping.loc[i, 'cell_ycrd']} - {activity_wd_ping.loc[i, 'cell_xcrd']} - {activity_wd_ping.loc[i, 'adng_nm']}"
    
    color='lime'
    size=2
    if activity_wd_ping.loc[i, 'time_zone'] == 4:
        color='darkorange'
        size=4
    
    folium.CircleMarker(location=[sub_lat, sub_long],
                        radius=size,
                        color=color,
                        tooltip=title).add_to(map)

map

Output hidden; open in https://colab.research.google.com to view.

### 주말

- DOW(요일) : 6, 7

In [ ]:
activity_wk = activity[((activity['time_zone'] == 3) | (activity['time_zone'] == 4)) & (activity['dow'] > 5)]
activity_wk.head()

,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
31,20220612,7,4146,용인시,41463540,기흥동,84343022,127.08938,37.230213,MALE,5,하이차저,4,3.619
47,20220619,7,4146,용인시,41461253,모현읍,88966963,127.25193,37.334324,MALE,10,EV라운지,4,4.235
66,20220626,7,4146,용인시,41463590,보정동,84999167,127.11206,37.314117,MALE,14,EV라운지,4,4.366
67,20220619,7,4146,용인시,41465540,죽전1동,85287185,127.12213,37.330376,FEMALE,5,EV라운지,4,4.154
71,20220625,6,4146,용인시,41465510,풍덕천1동,84599223,127.09788,37.328040,FEMALE,12,EV라운지,3,7.914


#### adng_nm 기준으로 그룹화

- geojson 동기준으로 나누어져있음 -> 이 파일도 동기준으로 그룹화하여 카운트
- 어느동에서 사용자가 앱을 많이 실행했는지 알 수 있는 지표가 됨

In [ ]:
activity_wk_group = activity_wk.groupby('adng_nm')['count_cust'].sum()
activity_wk_group = activity_wk_group.reset_index()
activity_wk_group = activity_wk_group.rename(columns={'count_cust' : 'count_cust_sum'})
activity_wk_group = activity_wk_group[['count_cust_sum']]
activity_wk_group.head()

,count_cust_sum
0,7756.272
1,6682.446
2,7664.890
3,7214.728
4,5225.809


#### activity_wK 데이터 중복 제거

- adng_nm 기준으로 데이터 중복 제거


In [ ]:
activity_wk_col = activity_wk.drop_duplicates(['adng_nm'])
activity_wk_col = activity_wk_col.sort_values('adng_nm')
activity_wk_col = activity_wk_col.reset_index()
activity_wk_col.head()

,index,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
0,1124,20220626,7,4146,용인시,41463520,구갈동,85447041,127.128075,37.270000,MALE,14,EV라운지,3,3.366
1,713,20220625,6,4146,용인시,41463560,구성동,85367096,127.125145,37.292526,FEMALE,8,EV라운지,4,12.147
2,31,20220612,7,4146,용인시,41463540,기흥동,84343022,127.089380,37.230213,MALE,5,하이차저,4,3.619
3,465,20220605,7,4146,용인시,41461259,남사읍,85894736,127.144455,37.145206,MALE,13,EV라운지,4,19.356
4,180,20220604,6,4146,용인시,41463572,동백1동,86599004,127.168610,37.285896,FEMALE,10,EV라운지,4,10.887


#### 병합

- 그룹화한 데이터프레임과 중복제거한 데이터프레임 붙이기

In [ ]:
activity_wk_col['count_cust_sum'] = activity_wk_group
activity_wk_col.head()

,index,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust,count_cust_sum
0,1124,20220626,7,4146,용인시,41463520,구갈동,85447041,127.128075,37.270000,MALE,14,EV라운지,3,3.366,7756.272
1,713,20220625,6,4146,용인시,41463560,구성동,85367096,127.125145,37.292526,FEMALE,8,EV라운지,4,12.147,6682.446
2,31,20220612,7,4146,용인시,41463540,기흥동,84343022,127.089380,37.230213,MALE,5,하이차저,4,3.619,7664.890
3,465,20220605,7,4146,용인시,41461259,남사읍,85894736,127.144455,37.145206,MALE,13,EV라운지,4,19.356,7214.728
4,180,20220604,6,4146,용인시,41463572,동백1동,86599004,127.168610,37.285896,FEMALE,10,EV라운지,4,10.887,5225.809


#### cell_id, time_zone기준으로 중복 제거

- 한 곳에 여러개가 찍혀도 확인을 할 수 없기 때문에 time_zone 별 한 곳에 하나만 찍히게 하기 위해서 중복 제거

In [ ]:
activity_wk_ping = activity_wk.drop_duplicates(['cell_id', 'time_zone'])
activity_wk_ping = activity_wk_ping.reset_index()
activity_wk_ping.head()

,index,base_dt,dow,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
0,31,20220612,7,4146,용인시,41463540,기흥동,84343022,127.08938,37.230213,MALE,5,하이차저,4,3.619
1,47,20220619,7,4146,용인시,41461253,모현읍,88966963,127.25193,37.334324,MALE,10,EV라운지,4,4.235
2,66,20220626,7,4146,용인시,41463590,보정동,84999167,127.11206,37.314117,MALE,14,EV라운지,4,4.366
3,67,20220619,7,4146,용인시,41465540,죽전1동,85287185,127.12213,37.330376,FEMALE,5,EV라운지,4,4.154
4,71,20220625,6,4146,용인시,41465510,풍덕천1동,84599223,127.09788,37.328040,FEMALE,12,EV라운지,3,7.914


#### 시각화

In [ ]:
map = folium.Map((37.240608, 127.1772935), zoom_start=11)
choropleth = folium.Choropleth(json_yongin,
                  data=activity_wk_col,
                  columns=['adng_nm', 'count_cust_sum'],
                  key_on='feature.id').add_to(map)


choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['adm_nm'], lables=False)
)

for i in activity_wk_ping.index:
    sub_lat = activity_wk_ping.loc[i, 'cell_ycrd']
    sub_long = activity_wk_ping.loc[i, 'cell_xcrd']
    
    title = f"{activity_wk_ping.loc[i, 'cell_ycrd']} - {activity_wk_ping.loc[i, 'cell_xcrd']} - {activity_wk_ping.loc[i, 'adng_nm']}"
    
    color='lime'
    size=2
    if activity_wk_ping.loc[i, 'time_zone'] == 4:
        color='darkorange'
        size=4
    
    folium.CircleMarker(location=[sub_lat, sub_long],
                        radius=size,
                        color=color,
                        tooltip=title).add_to(map)

map

Output hidden; open in https://colab.research.google.com to view.

# 3.ev_app_activity_resident.csv

In [ ]:
activity_resident = pd.read_csv('/content/drive/MyDrive/딥러닝/프로젝트/파일/3.ev_app_activity_resident.csv', encoding='utf-8')
activity_resident.head()

,base_dt,dow,mega_cd,mega_nm,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
0,20220611,6,41,경기도,4146,용인시,41461253,모현읍,86695054,127.171880,37.311146,FEMALE,13,EV라운지,1,4.492
1,20220605,7,11,서울특별시,1168,강남구,11680580,삼성1동,83591688,127.061264,37.509087,MALE,6,EV라운지,4,3.666
2,20220602,4,11,서울특별시,1120,성동구,11200670,성수2가1동,83463765,127.056560,37.540170,MALE,10,EV라운지,4,3.247
3,20220608,3,41,경기도,4146,용인시,41463510,신갈동,84951117,127.110490,37.290226,FEMALE,13,EV라운지,3,6.664
4,20220608,3,41,경기도,4150,이천시,41500330,호법면,92774456,127.386450,37.213290,MALE,11,EV라운지,5,3.945


#### activity_resident에서 원하는 컬럼만 뽑기

- ccw_nm가 용인시가 아닌 것만 추출

In [ ]:
activity_resident_1 = activity_resident[activity_resident['ccw_nm'] != '용인시']
activity_resident_1 = activity_resident_1.reset_index()
activity_resident_1.head()

,index,base_dt,dow,mega_cd,mega_nm,ccw_cd,ccw_nm,adng_cd,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender,age,app_web,time_zone,count_cust
0,1,20220605,7,11,서울특별시,1168,강남구,11680580,삼성1동,83591688,127.061264,37.509087,MALE,6,EV라운지,4,3.666
1,2,20220602,4,11,서울특별시,1120,성동구,11200670,성수2가1동,83463765,127.056560,37.540170,MALE,10,EV라운지,4,3.247
2,4,20220608,3,41,경기도,4150,이천시,41500330,호법면,92774456,127.386450,37.213290,MALE,11,EV라운지,5,3.945
3,5,20220615,3,11,서울특별시,1156,영등포구,11560540,여의동,79703961,126.923690,37.522020,MALE,9,EV라운지,1,8.104
4,11,20220605,7,41,경기도,4113,성남시,41133570,상대원1동,86743323,127.173050,37.433735,MALE,5,EV라운지,4,32.604


#### 시각화

In [ ]:
map = folium.Map((37.240608, 127.1772935), zoom_start=11)

folium.GeoJson(json_yongin).add_to(map)

for i in activity_resident_1.index:
    sub_lat = activity_resident_1.loc[i, 'cell_ycrd']
    sub_long = activity_resident_1.loc[i, 'cell_xcrd']
    
    title = f"{activity_resident_1.loc[i, 'cell_ycrd']} - {activity_resident_1.loc[i, 'cell_xcrd']} - {activity_resident_1.loc[i, 'adng_nm']}"
    
  
    color='seagreen'
    
    folium.CircleMarker(location=[sub_lat, sub_long],
                        radius=2,
                        color=color,
                        tooltip=title).add_to(map)

map

#### activity_resident에서 원하는 컬럼만 뽑기

- ccw_nm 용인시만 추출

In [ ]:
activity_resident_2 = activity_resident[activity_resident['ccw_nm'] == '용인시']
activity_resident_2 = activity_resident_2.reset_index()
activity_resident_2.head()

In [ ]:
map = folium.Map((37.240608, 127.1772935), zoom_start=11)

folium.GeoJson(json_yongin).add_to(map)

for i in activity_resident_2.index:
    sub_lat = activity_resident_2.loc[i, 'cell_ycrd']
    sub_long = activity_resident_2.loc[i, 'cell_xcrd']
    
    title = f"{activity_resident_2.loc[i, 'cell_ycrd']} - {activity_resident_2.loc[i, 'cell_xcrd']} - {activity_resident_2.loc[i, 'adng_nm']}"
    
  
    color='mediumpurple'
    
    folium.CircleMarker(location=[sub_lat, sub_long],
                        radius=2,
                        color=color,
                        tooltip=title).add_to(map)

map